In [1]:
GOOGLE_COLAB = False
if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    __dir__ = '/content/drive/My Drive/Colab Notebooks/nlp_ner_workshop/'
else:
    from os.path import abspath
    __dir__ = abspath("../")

## Reading the data

In [2]:
from zipfile import ZipFile
import json
import collections
import numpy as np
from keras.utils import to_categorical
from keras.layers import TimeDistributed, Dense, Activation, Input
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D, MaxPooling1D
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from matplotlib import pyplot as plt
from keras.layers import Dense, Activation, Dropout, Flatten

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

DATA_FILE = __dir__ + '/data/0.zip'



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [120]:
import pandas as pd

In [121]:
df = pd.DataFrame()


Our data consists of legal documents that have been pre-processed, tokenized and tagged. They're stored in one zip file of JSON files. To save space we'll read the files directly from the zip.
Let's see what one of them looks like.

In [153]:
import pandas as pd 
with ZipFile(DATA_FILE) as z:
    filename = z.filelist[0]
    with z.open(filename) as f:
        content = json.load(f)
        print(type(content),*content, sep="\n")
        

We can see that the document is split into lines (sequences) made of (token, tag) pairs. For instance, the tokens 'execution' and 'version' (3rd line) are tagged with 'b' indicating that they are **bold**.

Next, we'll concatenate all of these sequences into one big dataset.  
**NOTE**: Due to memory/time constraints we will limit the number of documents we use for training.

In [4]:
def read_json_zip_file(input_file, read_limit=1000):
    with ZipFile(input_file) as z:
        all_x = []
        for fname in z.filelist[:read_limit]:
            with z.open(fname) as f:
                all_x += json.load(f)
    return all_x

In [5]:
sequences = read_json_zip_file(DATA_FILE)

Since our CNN will work on sequences, we need to determine the maximum sequence length we'll use. The length of the sequence will impact the memory our model uses and the time it takes to train it.   
We'll choose a maximum sequence length of 256 so that almost 98% of our data will be used in the model.

In [155]:
def filter_sequences(sequences, max_length=256):
    lengths = [len(x) for x in sequences]
    print(f'Maximum length: {max(lengths)}')
    print(f'Minimum length: {min(lengths)}')
    print(f'Average length: {sum(lengths)/len(lengths)}')
    
    short_sequences = [s for s in sequences if len(s) <= max_length]
    print(f'% of short sequences: {100 * len(short_sequences)/len(sequences)}')
    X = [[c[0] for c in x] for x in short_sequences]
    y = [[c[1] for c in y] for y in short_sequences]
#     for i, y in enumerate(short_sequences[:100]):
#         print(y)
#         for c in y:
#             print(c[1])
    return X, y

In [156]:
max_length = 256
X, y = filter_sequences(sequences, max_length)

In [123]:
type(y)

list

## Encoding the sequences

Now that we have our word sequences ready we need to represent them in a machine readable (=numerical) format. To do that, we'll merge all of the tokens into one big corpus and assign each token a unique number.  
**NOTE**: We will remove words that appear less than `min_token_frequency` since we can't learn much from them.

In [62]:
min_token_frequency = 2
corpus = (token for sequence in X for token in sequence)

## Reserve the first two indexes for padding and uknowns
index2token = ["{pad}", "{unk}"] + [token for token, count in collections.Counter(corpus).items() if count >= min_token_frequency]
token2index = collections.defaultdict(lambda: 1, {token: index for index, token in enumerate(index2token)})

index2label = ["{pad}"] + list(set([label for target in y for label in target]))
label2index = {label: index for index, label in enumerate(index2label)}

with open(__dir__+'/model/model_params.json', 'w') as file:
    json.dump({
        "word2ind": dict(token2index),
        "label2ind": dict(label2index),
        "max_length": max_length
    }, file)

Let's see some stats about our data

In [9]:
max_sequence_length = max([len(x) for x in X])

print('Text vocabulary size: ', len(token2index))
print('Label vocabulary size: ', len(label2index))
print('Maximum sequence length: ', max_sequence_length)

Text vocabulary size:  23507
Label vocabulary size:  6
Maximum sequence length:  256


Next, we will use the dictionaries we've just created to encode our data.  
**NOTE**: We pad our data so that each sample we feed our network has the same size. (This is not necessarily required but it improves Keras' perfromance)

In [10]:
X_enc = [[token2index[token] for token in sequence] for sequence in X]
max_label = len(label2index)
y_enc = [[0] * (max_sequence_length - len(target)) + [label2index[label] for label in target] for target in y]
y_enc = [to_categorical(target, max_label) for target in y_enc]

X_enc = pad_sequences(X_enc, maxlen=max_sequence_length)
y_enc = pad_sequences(y_enc, maxlen=max_sequence_length)

The last step in preparing the data is to split it into train/test set which we will later feed into our model.

In [132]:
y_enc[:10, 2, :]

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=int32)

In [11]:
test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=test_size, random_state=42)

In [12]:
X_train.shape

(215141, 256)

In [13]:
y_train.shape

(215141, 256, 6)

## Building the model

Before building our model, let's define some parameters which we'll need later

In [99]:
vocabulary_size = len(token2index)
tags_size = len(label2index)


#training params
batch_size = 2048
num_epochs = 3

#model parameters
num_filters = 64 
embed_dim = 128 
# weight_decay = 1e-4

To build our CNN, we're going to use Keras. 

In [100]:
vocabulary_size

23507

Next, we create the embedding layer. This layer takes our numerical representation of tokens and learns a (hopefully) meaningful embedding for each of them. The input dimension for this layer is determined by the size of our vocabulary while the output dimension (embedding size) is chosen arbitrarily.

In [101]:
tags_size

6

In [159]:
from keras.models import Sequential
from keras import layers


model = Sequential()

model.add(Embedding(vocabulary_size, embed_dim, input_length=max_sequence_length, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))

model.add(Dropout(0.5))
model.add(layers.Dense(tags_size, activation='relu'))
model.add(TimeDistributed(layers.Dense(tags_size)))

model.add(layers.Dense(6, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 256, 128)          3008896   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 256, 64)           57408     
_________________________________________________________________
dropout_10 (Dropout)         (None, 256, 64)           0         
_________________________________________________________________
dense_20 (Dense)             (None, 256, 6)            390       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 256, 6)            42        
_________________________________________________________________
dense_22 (Dense)             (None, 256, 6)            42        
Total params: 3,066,778
Trainable params: 57,882
Non-trainable params: 3,008,896
_____________________________________________________________

In [160]:
from keras.preprocessing.sequence import pad_sequences
padded_x = pad_sequences(X_train)
# padded_x = X_train

In [161]:
padded_x.shape

(215141, 256)

## Training the model

In [ ]:
r = model.fit(padded_x, y_train, epochs=num_epochs, verbose=1)


Epoch 1/3
 67840/215141 [========>.....................] - ETA: 5:03 - loss: 0.0650 - acc: 0.9785

## (TODO) Add explanation 

In [87]:
# params = {
#     'vocabulary_size' :  vocabulary_size,
#     'tags_size' : tags_size,

#     #training params
#     'batch_size'  : batch_size,
#     'num_epochs'  : num_epochs,
#     #model parameters
#     'num_filters' : num_filters,
#     'embed_dim'   : embed_dim
# }

# artifact_dict = {
#     'classifier': model,
#     'params': params,
#     'metadata': {'model_ver' : 'version 2 26 Feb 2019'}
# }

Let's fit the model using the training data (this might tke a while depending on your hardware)

In [ ]:
# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# accuracies
plt.plot(r.history['acc'], label='acc')
plt.plot(r.history['val_acc'], label='val_acc')
plt.legend()

## Evaluating the model

We will use several techniques to evaluate the results of our model. The first one is simply to use Keras's built-in `evaluate` function which will return the test loss - lower means better.

In [109]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f'Model loss: {score}')

71714/71714 [==============================] - 56s 783us/step
Model loss: [0.01733744607059518, 0.995382373895562]


The next metrics we'll look at are the accuracy and confusion matrix

In [110]:
## Takes our sequence predictions/true values and returns them as lists 
def reshape_predictions(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

In [116]:
y_pred = model.predict(X_train).argmax(2)


(215141, 256)

In [148]:
yh = y_train.argmax(2)

In [149]:
yh.shape

(215141, 256)

In [152]:
yh = y_train.argmax(2)
fyh, fpr = reshape_predictions(yh, y_pred)
print('Training accuracy:', accuracy_score(fyh, fpr))
print('Training confusion matrix:')
# print(confusion_matrix(fyh, fpr))
print(fyh[:10])

Training accuracy: 0.9590764007211577
Training confusion matrix:
[2, 4, 4, 4, 2, 2, 2, 2, 2, 2]


Our accuracy is fairly high which is what we want but it might also indicate that our model isn't doing what we think it's doing. The accuracy metric measures the percentage of correct predictions. So, in our case, 95% if our tokens were marked as 'n' and 5% as any other label then if our model predicted 'n' for **all** input it would have an accuracy of 95%.  
Looking at the confusion matrix helps us gain insight as to what kind of errors our model makes. The columns of the matrix represent the prediction made by the model while the rows represent the true value - numbers on the diagonal are correct predicitions.  

Using the confusion matrix is sometimes difficult when comparing between different models/model parameters especially when there are a lot of classes. Looking at the precision, recall and F1 score might be easier 

In [94]:
results = precision_recall_fscore_support(fyh, fpr)
print(f'Precision: {results[0][1:]}')
print(f'Recall:    {results[1][1:]}')
print(f'F1-Score:  {results[2][1:]}')
print(f'Support:   {results[3][1:]}')

Precision: [0.96766706 0.67897022 0.61751479 0.        ]
Recall:    [0.99840974 0.03647482 0.54749837 0.        ]
F1-Score:  [0.98279804 0.06923053 0.58040261 0.        ]
Support:   [5831118  120028  130415     907]


Up until now we've only evaluated our models performance on the training data but in most cases we're more interested to know how well our model generalizes and how it performs on the data that it hasn't seen before - our test data.  
Let's evaluate the model's performance on the test data the same way we did for the train data

In [112]:
y_pred_test = model.predict(X_test).argmax(2)
yh_test = y_test.argmax(2)
fyh_test, fpr_test = reshape_predictions(yh_test, y_pred_test)
print('Training accuracy:', accuracy_score(fyh_test, fpr_test))
print('Training confusion matrix:')
print(confusion_matrix(fyh_test, fpr_test))

Training accuracy: 0.958535704957579
Training confusion matrix:
[[      0   13461      46    4967       0]
 [      0 1939047     313    1847       0]
 [      0   34106     656    5784       0]
 [      0   23837     215   20029       0]
 [      0     198       0       0       0]]


In [113]:
results_test = precision_recall_fscore_support(fyh_test, fpr_test)
print(f'Precision: {results_test[0][1:]}')
print(f'Recall:    {results_test[1][1:]}')
print(f'F1-Score:  {results_test[2][1:]}')
print(f'Support:   {results_test[3][1:]}')

Precision: [0.96438861 0.53333333 0.61387808 0.        ]
Recall:    [0.99888729 0.01617915 0.4543681  0.        ]
F1-Score:  [0.98133485 0.03140559 0.52221411 0.        ]
Support:   [1941207   40546   44081     198]


These results seem similar to the ones we got for the training data which means our model generalizes well.  
At this point we can save our model architecture and weights so that we can use it later without having to build and train it again.

In [114]:
with open(__dir__+'/model/model_architecture.json', 'w') as f:
    f.write(model.to_json())

model.save_weights(__dir__+'/model/model_weights.h5')